In [8]:
import pandas as pd
import requests
from io import StringIO

def get_withdrawn_ipos() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:

def classify_company(name: str) -> str:
    name_lc = name.lower()
    tokens = name_lc.replace('.', '').split()
    # Order matters: first match wins
    if 'acquisition' in tokens and ('corp' in tokens or 'corporation' in tokens):
        return 'Acq.Corp'
    if 'incorporated' in tokens or 'inc' in tokens:
        return 'Inc'
    if 'group' in tokens:
        return 'Group'
    if 'limited' in tokens or 'ltd' in tokens:
        return 'Limited'
    if 'holdings' in tokens:
        return 'Holdings'
    return 'Other'

In [9]:
tables = get_withdrawn_ipos()

In [12]:
tables

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


In [15]:
tables['Company Class'] = tables['Company Name'].apply(classify_company)


In [16]:
tables

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [17]:

# 3. Parse Price Range → Avg. price
import re

def parse_price_range(pr: str):
    # '$8.00-$10.00' → 9.0, '$5.00' → 5.0, '-' or missing → None
    if not isinstance(pr, str) or pr.strip() == '-' or pr.strip() == '':
        return None
    parts = pr.replace('$','').split('-')
    try:
        nums = [float(p) for p in parts if p]
        return sum(nums) / len(nums)
    except ValueError:
        return None

# Create Avg. Price column
if 'Price Range' in tables.columns:
    tables['Avg. Price'] = tables['Price Range'].apply(parse_price_range)
else:
    raise KeyError('Price Range column not found')

# 4. Clean Shares Offered → numeric
# Remove commas, handle missing
if 'Shares Offered' in tables.columns:
    tables['Shares Offered'] = (
        tables['Shares Offered']
        .astype(str)
        .str.replace(',', '')
        .str.extract(r"([0-9\.]+)")
        .astype(float)
    )
else:
    raise KeyError('Shares Offered column not found')

# 5. Compute Withdrawn Value
# Only where both Shares Offered and Avg. Price are not null
mask = tables['Shares Offered'].notna() & tables['Avg. Price'].notna()
tables.loc[mask, 'Withdrawn Value'] = tables.loc[mask, 'Shares Offered'] * tables.loc[mask, 'Avg. Price']

# 6. Group by Company Class and sum
grouped = (
    tables.dropna(subset=['Withdrawn Value'])
      .groupby('Company Class')['Withdrawn Value']
      .sum()
      .sort_values(ascending=False)
)

# 7. Output results
print("Total withdrawn IPO value by Company Class (in $ millions):")
print(grouped)

# Highest total
top_class = grouped.index[0]
top_value = grouped.iloc[0]
print(f"\nTop class: {top_class}, Total Withdrawn Value: ${top_value:.2f} million")

Total withdrawn IPO value by Company Class (in $ millions):
Company Class
Acq.Corp    4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Limited     5.497346e+08
Holdings    7.500000e+07
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64

Top class: Acq.Corp, Total Withdrawn Value: $4021000000.00 million


# Q2

In [39]:
import pandas as pd
import requests
from io import StringIO
import yfinance as yf

# 1. Fetch 2024 IPO list
def get_ipo_list(year: int) -> pd.DataFrame:
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    html_io = StringIO(resp.text)
    tables = pd.read_html(html_io)
    if not tables:
        raise ValueError(f"No IPO table found for {year}")
    return tables[0]

# Load and filter first-5-months 2024 IPOs
ipos_2024 = get_ipo_list(2024)
# Ensure there's a Date or "Date Filed" column



In [40]:
ipos_2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$3.45,-10.00%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$19.74,371.50%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.60,-85.00%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.25,2.50%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.31,3.10%
...,...,...,...,...,...,...
220,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$1.04,-82.67%
221,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.00,-15.32%
222,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$1.94,-57.00%
223,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$18.90,-7.48%


In [41]:
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'])
cutoff = pd.Timestamp('2024-06-01')
first5 = ipos_2024[ipos_2024['IPO Date'] < cutoff].copy()

In [43]:
print(f"First 5 months IPOs: {len(first5)} entries")  # expect 75


First 5 months IPOs: 77 entries


In [63]:
first5 = first5[(first5['IPO Price'].astype(str) != '-') &
                     (first5['Return'].astype(str) != '-')]

In [64]:
first5

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.15,113.88%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$20.13,2.91%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.52,5.15%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.30,-92.14%
153,2024-05-15,RAY,Raytech Holding Limited,$4.00,$1.18,-69.00%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.04,-82.67%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.00,-15.32%
222,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$1.94,-57.00%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$18.90,-7.48%


In [85]:

# Extract tickers
tickers = first5['Symbol'].dropna().unique().tolist()
print(f"Unique tickers: {len(tickers)}")

# 2. Download OHLCV data with yfinance
# Define date range from IPO date to 2025-06-06
start_date = first5['IPO Date'].min().strftime('%Y-%m-%d')
end_date = '2025-06-06'

Unique tickers: 75


In [86]:
start_date

'2024-01-09'

In [87]:
# Bulk download
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    group_by='Symbol',
    auto_adjust=False,
    progress=False
)

# 3. Compute growth and volatility for each ticker
def compute_metrics(df, periods):
    res = {}
    df = df['Close']
    # Daily returns for volatility: use pct_change()
    daily_ret = df.pct_change().dropna()
    res['volatility'] = daily_ret.std() * (252 ** 0.5)
    for p in periods:
        # price now vs p days ago
        res[f'growth_{p}d'] = df.iloc[-1] / df.shift(p).iloc[-1]  
    return res

metrics = []
periods = [1, 5, 21, 63, 126, 252]
for ticker in tickers:
    try:
        df_t = data[ticker].dropna()
        if len(df_t) < max(periods):
            continue
        m = compute_metrics(df_t, periods)
        m['Symbol'] = ticker
        metrics.append(m)
    except Exception:
        continue

stocks_df = pd.DataFrame(metrics).set_index('Symbol')


In [88]:
stocks_df

,volatility,growth_1d,growth_5d,growth_21d,growth_63d,growth_126d,growth_252d
Symbol,,,,,,,
BOW,0.353620,0.979070,0.969322,0.878965,1.068209,0.989289,1.352102
HDL,0.636071,1.019858,1.029464,0.997073,0.803143,1.039146,1.026620
JDZG,2.461775,0.604240,0.656430,0.570000,0.542857,0.499270,0.027426
RAY,1.749823,0.991453,0.397260,1.171717,1.234043,0.846715,0.315646
BTOC,1.138515,0.928962,1.000000,1.067265,0.708333,0.209139,0.237525
...,...,...,...,...,...,...,...
CCTG,1.614167,0.981982,0.908333,0.838462,0.654655,0.619318,0.497717
PSBD,0.208264,0.996405,1.003621,1.090480,0.898833,0.861405,0.849265
SYNX,1.024116,0.991354,0.982857,0.966292,0.452751,0.661538,0.656489


In [89]:
stocks_df["growth_252d"].isnull().sum()

np.int64(0)

In [90]:

# 4. Compute Sharpe
rf = 0.045
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - rf) / stocks_df['volatility']

# 5. Filter to date '2025-06-06'
# Our metrics are as of end of data (2025-06-06)
# 6. Descriptive stats
desc = stocks_df[['growth_252d', 'Sharpe']].describe()
print(desc)


       growth_252d     Sharpe
count    71.000000  71.000000
mean      1.143097   3.972258
std       1.445792   9.660363
min       0.024531  -0.011992
25%       0.280122   0.191887
50%       0.739311   0.667839
75%       1.200012   2.376179
max       8.962585  51.722663


In [91]:
stocks_df['growth_252d'].median()


np.float64(0.739311460919952)

In [92]:

# Median Sharpe ratio
median_sharpe = stocks_df['Sharpe'].median()
print(f"Median Sharpe (n={len(stocks_df)}): {median_sharpe:.4f}")

Median Sharpe (n=71): 0.6678


In [93]:


# 7. Top 10 by growth_252d vs Sharpe
top_growth = stocks_df.sort_values('growth_252d', ascending=False).head(10).index.tolist()
top_sharpe = stocks_df.sort_values('Sharpe', ascending=False).head(10).index.tolist()
print("Top 10 by growth_252d:", top_growth)
print("Top 10 by Sharpe:", top_sharpe)

# Compare lists
common = set(top_growth).intersection(top_sharpe)
print(f"Common tickers in both top lists: {sorted(common)}")

Top 10 by growth_252d: ['JL', 'ROMA', 'NNE', 'UMAC', 'RBRK', 'MFI', 'AS', 'AHR', 'MRX', 'MTEN']
Top 10 by Sharpe: ['BKHA', 'LEGT', 'IBAC', 'JVSA', 'DYCQ', 'AHR', 'HLXB', 'ULS', 'MRX', 'RBRK']
Common tickers in both top lists: ['AHR', 'MRX', 'RBRK']
